In [1]:
import pandas as pd

from ml import (list_all_experiments)




experiment_skill_map = {}


HIDE_SKILLS = True

# Get and print all experiments
all_experiments = list_all_experiments()
for exp_id, exp_name in all_experiments:
    if '2' in exp_name:
        continue
    if HIDE_SKILLS:
        if int(exp_id) < 27:
            continue
    skill = exp_name.split('Experience Model')[0]
    experiment_skill_map[skill] = exp_id
    
experiment_skill_map

You have 101 valid experiments & 25 invalid experiments & 1 metrics were missing.


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.Accuracy Class 0,metrics.Mean Accuracy,metrics.Recall Class 0,metrics.Recall Class 1,...,params.PCA_n_components,params.Sampling,params.Classifier,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.source.git.commit,tags.mlflow.runName,tags.mlflow.user,experiment_name
0,7d143c283d5249d38db2839572a90cf7,123,FINISHED,/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,2024-05-19 20:05:43.765000+00:00,2024-05-19 20:06:01.348000+00:00,0.917,0.912,0.958781,0.864253,...,150,SMOTE,LGBMClassifier,LOCAL,"[{""run_id"": ""7d143c283d5249d38db2839572a90cf7""...",/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,adf01c965de28b9ccf75451e73c6d5812f70f908,abrasive-mole-846,ez,Zulrah Experience Model comparison for 1000 users
1,eb549f9f65684131bc21fe1d2992f547,122,FINISHED,/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,2024-05-19 20:01:15.242000+00:00,2024-05-19 20:01:33.230000+00:00,0.873,0.873,0.892344,0.774390,...,150,SMOTE,SVM,LOCAL,"[{""run_id"": ""eb549f9f65684131bc21fe1d2992f547""...",/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,adf01c965de28b9ccf75451e73c6d5812f70f908,legendary-toad-32,ez,Zalcano Experience Model comparison for 1000 u...
2,677324049381435db577fbd2398e2360,121,FINISHED,/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,2024-05-19 19:56:56.647000+00:00,2024-05-19 19:57:15.103000+00:00,0.739,0.739,0.745679,0.710526,...,150,SMOTE,SVM,LOCAL,"[{""run_id"": ""677324049381435db577fbd2398e2360""...",/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,adf01c965de28b9ccf75451e73c6d5812f70f908,rare-stoat-906,ez,Wintertodt Experience Model comparison for 100...
3,056c62aacedf4a9cb39c22f1e2e65eca,120,FINISHED,/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,2024-05-19 19:52:53.458000+00:00,2024-05-19 19:53:14.275000+00:00,0.869,0.870,0.877647,0.862609,...,150,SMOTE,LogisticRegression,LOCAL,"[{""run_id"": ""056c62aacedf4a9cb39c22f1e2e65eca""...",/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,adf01c965de28b9ccf75451e73c6d5812f70f908,enchanting-crane-244,ez,Vorkath Experience Model comparison for 1000 u...
4,85e9dca781f44c6294a059bdaa7bda04,119,FINISHED,/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,2024-05-19 19:48:43.983000+00:00,2024-05-19 19:49:00.917000+00:00,0.899,0.899,0.923833,0.790323,...,150,SMOTE,SVM,LOCAL,"[{""run_id"": ""85e9dca781f44c6294a059bdaa7bda04""...",/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,adf01c965de28b9ccf75451e73c6d5812f70f908,intrigued-fawn-13,ez,Vet'ion Experience Model comparison for 1000 u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,95b78d041501413da42233dc1167f842,7,FINISHED,/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,2024-05-18 19:24:34.910000+00:00,2024-05-18 19:24:48.323000+00:00,0.876,0.876,0.893431,0.838095,...,100,SMOTE,LogisticRegression,LOCAL,"[{""run_id"": ""95b78d041501413da42233dc1167f842""...",/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,afe3434b6267eab2cb50688a4c30a95a0ff55907,silent-newt-822,ez,Hitpoints Experience Model comparison for 1000...
97,606efc9425d14d5d84d31bc1e0a8bcbc,6,FINISHED,/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,2024-05-18 19:10:37.313000+00:00,2024-05-18 19:10:48.808000+00:00,0.886,0.886,0.888430,0.879562,...,60,SMOTE,SVM,LOCAL,"[{""run_id"": ""606efc9425d14d5d84d31bc1e0a8bcbc""...",/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,afe3434b6267eab2cb50688a4c30a95a0ff55907,merciful-ox-55,ez,Strength Experience Model comparison for 1000 ...
98,64ee6846bc164aa59df0bdd05402f60b,5,FINISHED,/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,2024-05-18 19:07:23.958000+00:00,2024-05-18 19:08:24.574000+00:00,0.909,0.907,0.960598,0.765152,...,150,SMOTE,GradientBoosting,LOCAL,"[{""run_id"": ""64ee6846bc164aa59df0bdd05402f60b""...",/mnt/c/Users/Lounge/PyCharmProjects/osrs/pca A...,afe3434b6267eab2cb50688a4c30a95a0ff55907,gaudy-carp-919,ez,Defence Experience Model comparison for 1000 u...
99,1

In [9]:
def fetch_experiment_data(experiment_id):
    # Fetch all runs from a specific experiment
    runs = mlflow.search_runs(experiment_id)
    
    # Check if DataFrame is empty
    if runs.empty:
        print("No data found for experiment ID:", experiment_id)
        return None
    
    return runs

def get_best_experiment(experiment_id):
    try:
        experiment_data = fetch_experiment_data(str(experiment_id))
        max_roc_auc_row = experiment_data.loc[experiment_data['metrics.ROC-AUC'] == experiment_data['metrics.ROC-AUC'].max()]
        return max_roc_auc_row
    except:
        return None
max_roc_auc_row = get_best_experiment('3')

In [8]:
params = ['params.Classifier', 'params.Sampling', 'params.PCA_n_components']
max_roc_auc_row

In [3]:

from mlflow.tracking import MlflowClient

# Initialize MLflow client
client = MlflowClient()

# Load the model from a specific run

def finalize_experiement(run_id, skill_name):
    artifact_path = "model"
    print(run_id)
    model_uri = f"runs:/{run_id}/{artifact_path}"
    model = mlflow.pyfunc.load_model(model_uri)
    
    # Register the model in the Model Registry
    model_name = f'{skill_name} + extra features'
    model_details = mlflow.register_model(model_uri=model_uri, name=model_name)
    
    # Transition model to Production
    version = model_details.version
    client.transition_model_version_stage(
        name=model_name,
        version=version,
        stage="Production",
        archive_existing_versions=True
    )

    print(f"Model registered and transitioned to Production; version: {version}")



In [4]:
columns = []
rows = []
for skill , experiment_id in experiment_skill_map.items():
    max_roc_auc_row = get_best_experiment(experiment_id)
    if max_roc_auc_row is None:
        continue
    finalize_experiement(run_id=max_roc_auc_row['run_id'].tolist()[0], skill_name=skill)
    max_roc_auc_row['skill'] = skill
    rows.append(max_roc_auc_row)


No data found for experiment ID: 126
No data found for experiment ID: 125
No data found for experiment ID: 124
7d143c283d5249d38db2839572a90cf7


Registered model 'Zulrah  + extra features' already exists. Creating a new version of this model...
2024/05/22 11:22:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Zulrah  + extra features, version 3
Created version '3' of model 'Zulrah  + extra features'.
/tmp/ipykernel_10868/1537320839.py:20: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Model registered and transitioned to Production; version: 3
eb549f9f65684131bc21fe1d2992f547


Registered model 'Zalcano  + extra features' already exists. Creating a new version of this model...
2024/05/22 11:22:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Zalcano  + extra features, version 3
Created version '3' of model 'Zalcano  + extra features'.
/tmp/ipykernel_10868/1537320839.py:20: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Model registered and transitioned to Production; version: 3
677324049381435db577fbd2398e2360


...

Registered model 'Bounty Hunter - Rogue  + extra features' already exists. Creating a new version of this model...
2024/05/22 11:22:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Bounty Hunter - Rogue  + extra features, version 3
Created version '3' of model 'Bounty Hunter - Rogue  + extra features'.
/tmp/ipykernel_10868/1537320839.py:20: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Model registered and transitioned to Production; version: 3
00130a61f33b4f9881c02c741db98452


Registered model 'Bounty Hunter - Hunter  + extra features' already exists. Creating a new version of this model...
2024/05/22 11:22:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Bounty Hunter - Hunter  + extra features, version 3


Model registered and transitioned to Production; version: 3


Created version '3' of model 'Bounty Hunter - Hunter  + extra features'.
/tmp/ipykernel_10868/1537320839.py:20: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [5]:
df = pd.concat(rows)

columns = ['params.Classifier','skill','params.PCA_n_components','metrics.Mean Accuracy','metrics.ROC-AUC', 'metrics.Recall Class 1','metrics.Recall Class 0','metrics.Accuracy Class 1','metrics.Accuracy Class 0']

df = df[columns]

int_columns = ['params.PCA_n_components']
float_columns = ['metrics.Mean Accuracy','metrics.ROC-AUC','metrics.Recall Class 1','metrics.Recall Class 0','metrics.Accuracy Class 1','metrics.Accuracy Class 0']

float_columns = [col for col in float_columns if col in columns]
int_columns = [col for col in int_columns if col in columns]


df[int_columns] = df[int_columns].astype(int)
df[float_columns] = df[float_columns].astype(float)
df

,params.Classifier,skill,params.PCA_n_components,metrics.Mean Accuracy,metrics.ROC-AUC,metrics.Recall Class 1,metrics.Recall Class 0,metrics.Accuracy Class 1,metrics.Accuracy Class 0
1,LGBMClassifier,Zulrah,150,0.912,0.911517,0.864253,0.958781,0.917,0.917
3,SVM,Zalcano,150,0.873,0.833367,0.774390,0.892344,0.873,0.873
3,SVM,Wintertodt,150,0.739,0.728103,0.710526,0.745679,0.739,0.739
2,LogisticRegression,Vorkath,150,0.870,0.870128,0.862609,0.877647,0.869,0.869
3,SVM,Vet'ion,150,0.899,0.857078,0.790323,0.923833,0.899,0.899
3,SVM,Venenatis,150,0.889,0.821001,0.715084,0.926918,0.889,0.889
3,SVM,Theatre of Blood: Hard Mode,150,0.942,0.823596,0.689655,0.957537,0.942,0.942
10,SVM,Theatre of Blood,6,0.877,0.858279,0.835052,0.881506,0.877,0.877
22,LogisticRegression,Tempoross,50,0.840,0.798926,0.737805,0.860048,0.840,0.840
37,LogisticRegression,Spindel,40,0.917,0.884370,0.836735,0.932005,0.918,0.918


In [6]:
df[['skill', 'metrics.Mean Accuracy', 'metrics.Accuracy Class 1','metrics.Accuracy Class 0']]

,skill,metrics.Mean Accuracy,metrics.Accuracy Class 1,metrics.Accuracy Class 0
1,Zulrah,0.912,0.917,0.917
3,Zalcano,0.873,0.873,0.873
3,Wintertodt,0.739,0.739,0.739
2,Vorkath,0.870,0.869,0.869
3,Vet'ion,0.899,0.899,0.899
3,Venenatis,0.889,0.889,0.889
3,Theatre of Blood: Hard Mode,0.942,0.942,0.942
10,Theatre of Blood,0.877,0.877,0.877
22,Tempoross,0.840,0.840,0.840
37,Spindel,0.917,0.918,0.918
